In [ ]:
import pandas as pd

from glmpy import sim
from glmpy.nml import glm_nml, nml

## `NMLParam`

Class for individual NML parameters. `NMLParam` objects store the value, name, type, units, and validation logic for a parameter.

In [ ]:
min_layer_vol = glm_nml.NMLParam(
    name="min_layer_vol", 
    type=float, 
    value=0.5, 
    units="m^3", 
    val_gt=0.0
)

In [ ]:
min_layer_vol.value

In [ ]:
min_layer_vol.units

In [ ]:
min_layer_vol.value = -2.5

In [ ]:
min_layer_vol.validate()

In [ ]:
min_layer_vol.strict = False
min_layer_vol.value = -2.5
min_layer_vol.validate()

### `NMLBlock`

Abstract class for storing a collection of `NMLParams` and the block validation logic. Provides methods for returning a dictionary of parameter values (`get_nml_dict()`) and intra-block validation. To create a new block class for custom NML files, subclass `NMLBlock`. `glm_nml` provides classes for all the standard blocks, e.g., `GLMSetupBlock`, `MixingBlock`, and`MorphometryBlock`.

In [ ]:
class CustomBlock(glm_nml.NMLBlock):
    def __init__(
        self,
        string_param=None,
        bool_param=None,
        int_param=None,
        list_float_param=None
    ):
        super().__init__()
        self.params["string_param"] = glm_nml.NMLParam(
            "string_param", str, string_param, val_required=True
        )
        self.params["bool_param"] = glm_nml.NMLParam(
            "bool_param", bool, bool_param,
        )
        self.params["int_param"] = glm_nml.NMLParam(
            "int_param", int, int_param, val_switch=[1, 2, 3]
        )
        self.params["list_float_param"] = glm_nml.NMLParam(
            "list_float_param", float, list_float_param, "m^3", is_list=True
        )
        self.block_name = "custom_block"
        self.strict = True
    
    def validate(self):
        self.params.validate()
        self.val_list_len_params("int_param", "list_float_param")

In [ ]:
custom_block = CustomBlock(
    string_param="abc",
    bool_param=True,
    int_param=3,
    list_float_param=[9.9, 9.8, 9.7]
)
custom_block.get_block_dict()

In [ ]:
custom_block.strict = False
custom_block.params["int_param"].value = 4
custom_block.get_block_dict()

In [ ]:
glm_setup = glm_nml.GLMSetupBlock(
    sim_name="sparkling",
    max_layers=500,
    min_layer_vol=0.5,
    min_layer_thick=0.15,
    max_layer_thick=0.5,
    density_model=1,
    non_avg=True
)
time = glm_nml.TimeBlock(
    timefmt=3,
    start="1980-04-15",
    stop="2012-12-10",
    dt=3600.0,
    timezone=-6.0,
    num_days=730
)
morphometry = glm_nml.MorphometryBlock(
    lake_name="nhd_13344210",
    latitude=46.00881,
    longitude=-89.69953,
    bsn_len=901.0385,
    bsn_wid=901.0385,
    crest_elev=320.0,
    bsn_vals=16,
    H=[
        301.712,303.018285714286,304.324571428571,305.630857142857,
        306.937142857143,308.243428571429,309.549714285714,310.856,
        312.162285714286,313.468571428571,314.774857142857,316.081142857143,
        317.387428571429,318.693714285714,320.0,321.0
    ],
    A=[
        0.0,45545.8263571429,91091.6527142857,136637.479071429,
        182183.305428571,227729.131785714,273274.958142857,318820.7845,
        364366.610857143,409912.437214286,455458.263571429,501004.089928571,
        546549.916285714,592095.742642857,637641.569,687641.569
    ]
)
init_profiles = glm_nml.InitProfilesBlock(
    lake_depth=18.288,
    num_depths=3,
    the_depths=[0.0,0.2,18.288],
    the_temps=[3.0,4.0,4.0],
    the_sals=[0.0,0.0,0.0],
    num_wq_vars=6,
    wq_names=["OGM_don","OGM_pon","OGM_dop","OGM_pop","OGM_doc","OGM_poc"],
    wq_init_vals=[
        1.1,1.2,1.3,1.2,1.3,2.1,2.2,2.3,1.2,1.3,3.1,3.2,3.3,1.2,1.3,4.1,4.2,
        4.3,1.2,1.3,5.1,5.2,5.3,1.2,1.3,6.1,6.2,6.3,1.2,1.3
    ]
)

### `NML`

Abstract base class for storing a collection of `NMLBlock` objects and the inter-block validation logic. Provides methods for writing the NML (`write_nml()`), returing the NML as a dictionary (`get_nml_dict()`), and validating the NML (`validate()`). Subclass `NML` to create objects for specific NML files, e.g., `GLMNML`, `AEDNML`. 


In [ ]:
glm_nml_file = glm_nml.GLMNML(
    glm_setup=glm_setup,
    time=time,
    morphometry=morphometry,
    init_profiles=init_profiles

)
glm_nml_file.get_nml_dict()

In [ ]:
glm_nml_file.strict = False
glm_nml_file.blocks["custom_block"] = custom_block
glm_nml_file.get_nml_dict()

In [ ]:
glm_nml_file.strict = True
glm_nml_file.blocks["custom_block"].params["int_param"].value = 3
glm_nml_file.get_nml_dict()

In [ ]:
glm_nml_file.write_nml("custom_nml.nml")

### `GLMSim`

Class for defining and running GLM simulations. Simulation data, such as NML parameters and boundary condition data, are stored on the object rather than as file paths. Enables simulation-wide validation (between NML and boundary condition data), modifying  parameters/data on the simulation object, and sub-classing to create template simulations. 



In [ ]:
met_data_pd = pd.DataFrame()

glm_sim = sim.GLMSim(
    glm_nml=glm_nml_file,
    bcs={"met_data": met_data_pd},
    sim_name="glm_sim"
)

In [ ]:
glm_sim.to_file("custom_sim.glmpy")

In [ ]:
from glmpy.example_sims import SparklingSim

sparkling = SparklingSim()

In [ ]:
sparkling.bcs["nldas_driver"]

In [ ]:
sparkling.nml["glm"].get_nml_dict()

In [ ]:
sparkling.nml["glm"].blocks["meteorology"].params["meteo_fl"].value

In [ ]:
sparkling.get_param_value("glm", "meteorology", "meteo_fl")

In [ ]:
sparkling.nml["glm"].blocks["meteorology"].params["meteo_fl"].value = "abc/nldas_driver.csv"

In [ ]:
sparkling.set_param_value(
    "glm", "meteorology", "meteo_fl", "abc/nldas_driver.csv"
)

In [ ]:
sparkling.run()